In [21]:
import pandas as pd
import numpy as np
import re

class FertilizerStandardizer:
    def __init__(self):
        # Liste des engrais ICV
        self.icv_fertilizer_list = ["Solution Liquide N 39"]
        
        # Masses volumiques 
        self.default_mineral_density = 1.325  # kg/L pour minéraux liquides
        self.organic_solid_density = 1000     # kg/m³ pour organiques solides
        self.organic_liquid_density = 1.0     # kg/L pour organiques liquides

        # Mapping des types d'intervention vers type_engrais
        self.type_mapping = {
            # Mesparcelles
            "Fertilisation et amendement organique": "engrais organiques",
            "Fertilisation et amendement mineral - foliaire inclus": "engrais mineraux",
            # SMAG
            "Fertilisation organique": "engrais organiques",
            "Fertilisation minérale": "engrais mineraux"
        }

    def standardize_fertilizer_name(self, name):
        if pd.isna(name):
            return name
        name = str(name).upper()
        name = re.sub(r'[^\w\s]', ' ', name)
        name = re.sub(r'\s+', ' ', name)
        name = name.strip()
        return name

    def is_in_icv_list(self, fertilizer_name):
        std_name = self.standardize_fertilizer_name(fertilizer_name)
        std_icv_list = [self.standardize_fertilizer_name(name) for name in self.icv_fertilizer_list]
        return std_name in std_icv_list

    def standardize_type_engrais(self, intervention_type):
        """Standardise le type d'intervention en type_engrais"""
        if pd.isna(intervention_type):
            return "engrais mineraux"  # valeur par défaut
        
        type_std = self.type_mapping.get(intervention_type)
        if type_std is None:
            print(f"Avertissement: Type d'intervention non reconnu: {intervention_type}")
            return "engrais mineraux"  # valeur par défaut
        return type_std

    def convert_to_kg(self, row):
        try:
            if pd.isna(row.get('unite_intrant_intervention')) or pd.isna(row.get('quantite_totale')):
                return np.nan
                
            unite = str(row.get('unite_intrant_intervention', '')).upper()
            type_engrais = str(row.get('type_engrais', '')).lower()
            quantite = float(row.get('quantite_totale', 0))
            
            if unite == 'KG':
                return quantite
            elif unite == 'L':
                if 'organique' in type_engrais:
                    return quantite * self.organic_liquid_density
                else:  # engrais minéraux
                    return quantite * self.default_mineral_density
            elif unite == 'M3':
                if 'organique' in type_engrais:
                    return quantite * self.organic_solid_density
                else:  # engrais minéraux en m3
                    return quantite * self.default_mineral_density * 1000
            else:
                print(f"Avertissement: Unité non reconnue: {unite} pour l'engrais {row.get('libelle', 'Unknown')}")
                return np.nan
        except Exception as e:
            print(f"Erreur lors de la conversion pour l'engrais {row.get('libelle', 'Unknown')}: {str(e)}")
            return np.nan

    def convert_dose_to_kg_ha(self, row):
        try:
            if pd.isna(row.get('unite')) or pd.isna(row.get('dose')):
                return np.nan
                
            if row['unite'].upper() == 'KG/HA':
                return row['dose']
            elif row['unite'].upper() == 'L/HA':
                if row['type_engrais'] == 'engrais organiques':
                    return row['dose'] * self.organic_liquid_density
                else:
                    return row['dose'] * self.default_mineral_density
            elif row['unite'].upper() == 'M3/HA':
                if row['type_engrais'] == 'engrais organiques':
                    return row['dose'] * self.organic_solid_density
                else:
                    return row['dose'] * self.default_mineral_density * 1000
            else:
                print(f"Avertissement: Unité non reconnue: {row['unite']} pour l'engrais {row['libelle']}")
                return np.nan
        except Exception as e:
            print(f"Erreur lors de la conversion pour l'engrais {row['libelle']}: {str(e)}")
            return np.nan

    def standardize_mesparcelles(self, data):
        print("Début de la standardisation des données Mesparcelles...")
        
        # Filtrer d'abord pour ne garder que les fertilisations
        fertilisation_types = [
            "Fertilisation et amendement organique",
            "Fertilisation et amendement mineral - foliaire inclus"
        ]
        data = data[data['type_intervention.libelle'].isin(fertilisation_types)].copy()
        print(f"Après filtrage des types d'intervention: {len(data)} lignes")
        
        # Le reste du code reste identique...
        required_columns = [
            'libelle', 
            'unite_intrant_intervention', 
            'quantite_totale', 
            'surface_travaillee_ha', 
            'fertilisant.composition.n_total',
            'fertilisant.composition.p', 
            'fertilisant.composition.k',
            'type_intervention.libelle'
        ]
        
        # Continuer avec le reste de la méthode...
        df = data.dropna(subset=required_columns).copy()
        df['type_engrais'] = df['type_intervention.libelle'].apply(self.standardize_type_engrais)
        df['in_icv_list'] = df['libelle'].apply(self.is_in_icv_list)
        print("Conversion des quantités en kg...")
        df['quantite_kg'] = df.apply(self.convert_to_kg, axis=1)
        df['dose_kg_ha'] = df['quantite_kg'] / df['surface_travaillee_ha']
        print("Calcul des nutriments pour les engrais hors ICV...")
        df = df.apply(self.calculate_nutrients, axis=1)
        return self.aggregate_data(df)

    def standardize_smag(self, data):
        print("Début de la standardisation des données SMAG...")
    
        # Filtrer d'abord pour ne garder que les fertilisations
        fertilisation_types = [
        "Fertilisation organique",
        "Fertilisation minérale"
    ]
        data = data[data["Type d'intervention"].isin(fertilisation_types)].copy()
        print(f"Après filtrage des types d'intervention: {len(data)} lignes")
    
        # Colonnes requises pour SMAG avec les nouveaux noms
        required_columns = [
        'Intrant', 
        'Unité', 
        'Dose', 
        'Surface intervention sur parcelle',
        "Type d'intervention",
        'N',  # Ajout des colonnes NPK
        'P',
        'K'
    ]
    
        # Filtrer les lignes avec des valeurs non-NaN pour les colonnes requises
        df = data.dropna(subset=required_columns).copy()
    
        # Renommer les colonnes pour correspondre au format standard
        df = df.rename(columns={
        'Intrant': 'libelle',
        'Unité': 'unite',
        'Dose': 'dose',
        'Surface intervention sur parcelle': 'surface_travaillee_ha',
        'N': 'N_kg_ha',  # Ces valeurs sont déjà en kg/ha
        'P': 'P_kg_ha',
        'K': 'K_kg_ha'
        })
    
        # Convertir le type d'intervention en type_engrais
        df['type_engrais'] = df["Type d'intervention"].apply(self.standardize_type_engrais)
        
        # Vérification présence dans ICV
        df['in_icv_list'] = df['libelle'].apply(self.is_in_icv_list)
        
        # Conversion des doses en kg/ha seulement pour les engrais ICV
        print("Conversion des doses en kg/ha...")
        df.loc[df['in_icv_list'], 'dose_kg_ha'] = df[df['in_icv_list']].apply(self.convert_dose_to_kg_ha, axis=1)
        
        return self.aggregate_data(df)

    def calculate_nutrients(self, row):
        try:
            if not row['in_icv_list']:
                if row['type_engrais'] == 'engrais organiques':
                    volume_m3 = row['quantite_totale'] if row['unite_intrant_intervention'].upper() == 'M3' \
                        else row['quantite_kg'] / self.organic_solid_density
                    
                    row['N_kg_ha'] = row['fertilisant.composition.n_total'] * volume_m3 / row['surface_travaillee_ha']
                    row['P_kg_ha'] = row['fertilisant.composition.p'] * volume_m3 / row['surface_travaillee_ha']
                    row['K_kg_ha'] = row['fertilisant.composition.k'] * volume_m3 / row['surface_travaillee_ha']
                else:
                    row['N_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.n_total'] / 100
                    row['P_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.p'] / 100
                    row['K_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.k'] / 100
            return row
        except Exception as e:
            print(f"Erreur lors du calcul des nutriments pour l'engrais {row['libelle']}: {str(e)}")
            return row

    def aggregate_data(self, df):
        print("Agrégation des données...")
        agg_funcs = {
            'surface_travaillee_ha': 'sum'
        }
        
        if 'quantite_kg' in df.columns:
            agg_funcs['quantite_kg'] = 'sum'
            
        if 'dose_kg_ha' in df.columns:
            agg_funcs['dose_kg_ha'] = lambda x: np.average(x, weights=df.loc[x.index, 'surface_travaillee_ha'])
            
        for nutrient in ['N_kg_ha', 'P_kg_ha', 'K_kg_ha']:
            if nutrient in df.columns:
                agg_funcs[nutrient] = lambda x: np.average(x, weights=df.loc[x.index, 'surface_travaillee_ha'])
        
        return df.groupby(['libelle', 'type_engrais', 'in_icv_list']).agg(agg_funcs).reset_index()

    def combine_sources(self, mesparcelles_data, smag_data):
        print("Combinaison des données des deux sources...")
        mp_std = self.standardize_mesparcelles(mesparcelles_data)
        smag_std = self.standardize_smag(smag_data)
        
        mp_std['source'] = 'mesparcelles'
        smag_std['source'] = 'smag'
        
        return pd.concat([mp_std, smag_std], ignore_index=True)

# Import des données
print("Import des données...")

# Import Mesparcelles
mesparcelles_df = pd.read_excel(
    r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\mesparcelles_data.xlsx",
    sheet_name='Intervention_Details',
    engine='openpyxl'
)
print(f"Données Mesparcelles importées : {len(mesparcelles_df)} lignes")

# Import SMAG
smag_df = pd.read_excel(
    r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\SMAG.xlsx",
    engine='openpyxl'
)
print(f"Données SMAG importées : {len(smag_df)} lignes")

# Afficher les colonnes disponibles pour vérification
print("\nColonnes Mesparcelles :")
print(mesparcelles_df.columns.tolist())
print("\nColonnes SMAG :")
print(smag_df.columns.tolist())

# Vérifier les types d'intervention uniques
print("\nTypes d'intervention dans Mesparcelles :")
print(mesparcelles_df['type_intervention.libelle'].unique())
print("\nTypes d'intervention dans SMAG :")
print(smag_df["Type d'intervention"].unique())

# Traitement des données
standardizer = FertilizerStandardizer()
combined_data = standardizer.combine_sources(mesparcelles_df, smag_df)

# Affichage des résultats
print("\nRésultats pour les engrais dans la liste ICV (Solution Liquide N 39):")
icv_data = combined_data[combined_data['in_icv_list']]
if len(icv_data) > 0:
    print(icv_data[['libelle', 'dose_kg_ha', 'surface_travaillee_ha', 'source']])
else:
    print("Aucun engrais ICV trouvé")

print("\nRésultats pour les autres engrais (doses NPK):")
non_icv_data = combined_data[~combined_data['in_icv_list']]
if len(non_icv_data) > 0:
    print(non_icv_data[['libelle', 'N_kg_ha', 'P_kg_ha', 'K_kg_ha', 'surface_travaillee_ha', 'source']])
else:
    print("Aucun engrais non-ICV trouvé")

# Export des résultats
combined_data.to_excel('resultats_standardisation.xlsx', index=False)
print("\nRésultats exportés dans 'resultats_standardisation.xlsx'")

Import des données...
Données Mesparcelles importées : 14 lignes
Données SMAG importées : 34540 lignes

Colonnes Mesparcelles :
['uuid_intervention', 'numero_lot', 'siret_exploitation', 'uuid_parcelle', 'link_parcelle', 'surface_travaillee_ha', 'date_debut', 'date_fin', 'materiels', 'prestataires', 'culture.id_culture', 'culture.libelle', 'type_intervention.id_type_intervention', 'type_intervention.libelle', 'id_intrant', 'libelle', 'quantite_totale', 'unite_intrant_intervention', 'phyto', 'type_intrant.id_type_intrant', 'type_intrant.libelle', 'type_intrant.categorie', 'fertilisant.composition.n_total', 'fertilisant.composition.p', 'fertilisant.composition.k', 'fertilisant.composition.cao', 'fertilisant.composition.mgo', 'fertilisant.composition.s', 'fertilisant.composition.unite', 'fertilisant.condition_epandage', 'fertilisant.condition_epandage.id_condition_epandage', 'fertilisant.condition_epandage.libelle', 'fertilisant', 'phyto.code_amm', 'phyto.cible']

Colonnes SMAG :
['Code pr